Template notebook for one type of model.

In [1]:
import importlib
import sys
def reload_helper_functions():
    """Reload helper_functions.py if it is already imported."""
    if "helper_functions" in sys.modules:
        importlib.reload(sys.modules["helper_functions"])
    else:
        import helper_functions

In [2]:
import importlib
import sys
reload_helper_functions()

# General imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import helper_functions

# Model specific imports
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from xgboost import XGBClassifier,XGBRegressor
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from xgboost import XGBRegressor, XGBClassifier
from statistics import mean
from sklearn.preprocessing import LabelEncoder


In [3]:
# Load data
data = pd.read_csv("train_data_enriched.csv", index_col="Claim Identifier")
data_test = pd.read_csv("test_data_enriched.csv",index_col="Claim Identifier")

/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_10889/3771019286.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("train_data_enriched.csv", index_col="Claim Identifier")


In [4]:
# ------------------------

In [5]:
num_features = ['Age at Injury', 'Average Weekly Wage', 'Birth Year', 'IME-4 Count', 'Number of Dependents',
                "Accident Year","Accident Month","Accident Day","Accident DayOfWeek","Assembly Date DSA",
                "C-2 Date DSA","C-3 Date DSA","First Hearing Date DSA" ,"Accident Year",
    "Accident Month",
    "Accident Day",
    "Accident DayOfWeek","Accident Date","C-3 Date","First Hearing Date"]

In [6]:
cat_features = [
    "Alternative Dispute Resolution",
    "Carrier Name",
    "Carrier Type",
    "County of Injury",
    "District Name",
    "Gender",
    "Industry Code",
    "Medical Fee Region",
    "WCIO Cause of Injury Code",
    "WCIO Nature of Injury Code",
    "WCIO Part Of Body Code",
    "Age at Injury Category",
    "Carrier Claim Category",
    "Body Section"
]

In [7]:
selected_features = ['Accident Date', 'Age at Injury', 'Alternative Dispute Resolution', 'Attorney/Representative', 'Average Weekly Wage', 'C-3 Date', 'Carrier Name', 'Carrier Type', 'County of Injury',
                      'COVID-19 Indicator', 'District Name', 'First Hearing Date', 'Gender', 'IME-4 Count', 'Industry Code', 'Medical Fee Region', 'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code', 
                      'WCIO Part Of Body Code', 'Number of Dependents', 'Accident Year', 'Accident Month', 'Accident Day', 'Accident DayOfWeek', 'C-2 Date DSA', 'C-3 Date DSA',
                         'Age at Injury Category', # 'Accident Date_missing', 'First Hearing Date_missing', 'C-3 Date_missing', 'Assembly Date_missing', 'C-2 Date_missing',
                        'Carrier Claim Category', 'Body Section']

# Select Categorical features
selected_cat_features = ['Alternative Dispute Resolution', 'Attorney/Representative', 'Carrier Name', 'Carrier Type', 'County of Injury',
                        'COVID-19 Indicator', 'District Name', 'Gender', 'Industry Code', 'Medical Fee Region', 'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code',
                        'WCIO Part Of Body Code', 'Carrier Claim Category', 'Body Section'] # based on RFE

# Select numerical features
selected_num_features = ['Age at Injury', 'Average Weekly Wage','IME-4 Count','Number of Dependents',
                         'Accident Year','Accident Month', 'Accident Day', 'Accident DayOfWeek',
                         'C-2 Date DSA', 'C-3 Date DSA'] # based on RFE

In [8]:
# Dropping redundant variables that carry almost the same information (are extremely correlated (|0.8|))
# We believe it was better to keep Age at Injury than birth year since it should be more related to the injury claim type (it will be tested later)
# The same logic was applied to dropping the other two dates and two DSA variables since we believe Accident date to be more important
# Para `data`
data = data.loc[:, ~data.columns.isin(['Birth Year', 'Assembly Date', 'C-2 Date', 'Assembly Date DSA', 'First Hearing Date DSA'])]

# Para `data_test`
data_test = data_test.loc[:, ~data_test.columns.isin(['Birth Year', 'Assembly Date', 'C-2 Date', 'Assembly Date DSA', 'First Hearing Date DSA'])]


for col in ['Birth Year',"Assembly Date DSA", "First Hearing Date DSA"]:
    num_features.remove(col)


#Since the codes always seem to provide the same or more information than the descriptions (have more categories),
#and the codes are consistent (always only having 1 description for code, while descriptions may have multiple codes)
#we will drop the description columns.
data.drop(['Industry Code Description','WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description'], axis = 1,inplace = True)
data_test.drop(['Industry Code Description','WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description'], axis = 1,inplace = True)


# Removing Zip Code for reason meantion in pre-processement
data.drop(['Zip Code'], axis=1, inplace = True)
data_test.drop(['Zip Code'], axis=1 , inplace = True)

In [9]:
le = LabelEncoder()
#Label inconding our target variable 
data["Claim Injury Type"] = le.fit_transform(data["Claim Injury Type"])

In [10]:
X = data.drop(["Claim Injury Type"], axis = 1)
y = data["Claim Injury Type"]

In [ ]:
xgb_results = helper_functions.cv_scores(XGBClassifier(), X, y,num_features,cat_features,scalling_outlier =False)
xgb_results
# look at results

In [ ]:
# Test prediction
    # Choose best model from KFold above
    # Train model on whole train and predict test data
submission = helper_functions.test_prediction(XGBClassifier(),X,y,num_features,cat_features,data_test) 
submission

Sem dados suficientes para imputar a coluna 'Number of Dependents', usando mediana.
Sem dados suficientes para imputar a coluna 'Number of Dependents', usando mediana.


/Users/joaosantos/Documents/GitHub/DSAA_ML_41/code/models/helper_functions.py:63: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[target_column].fillna(data[target_column].median(), inplace=True)
/Users/joaosantos/Documents/GitHub/DSAA_ML_41/code/models/helper_functions.py:63: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we a

Sem dados suficientes para imputar a coluna 'Accident Year', usando mediana.
Sem dados suficientes para imputar a coluna 'Accident Year', usando mediana.
Sem dados suficientes para imputar a coluna 'Accident Month', usando mediana.
Sem dados suficientes para imputar a coluna 'Accident Month', usando mediana.
Sem dados suficientes para imputar a coluna 'Accident Day', usando mediana.
Sem dados suficientes para imputar a coluna 'Accident Day', usando mediana.
Sem dados suficientes para imputar a coluna 'Accident DayOfWeek', usando mediana.
Sem dados suficientes para imputar a coluna 'Accident DayOfWeek', usando mediana.


/Users/joaosantos/Documents/GitHub/DSAA_ML_41/code/models/helper_functions.py:63: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[target_column].fillna(data[target_column].median(), inplace=True)
/Users/joaosantos/Documents/GitHub/DSAA_ML_41/code/models/helper_functions.py:63: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we a

Sem dados suficientes para imputar a coluna 'Age at Injury', usando mediana.


...

Sem dados suficientes para imputar a coluna 'Accident Year', usando mediana.
Sem dados suficientes para imputar a coluna 'Accident Year', usando mediana.
Sem dados suficientes para imputar a coluna 'Accident Month', usando mediana.
Sem dados suficientes para imputar a coluna 'Accident Month', usando mediana.
Sem dados suficientes para imputar a coluna 'Accident Day', usando mediana.
Sem dados suficientes para imputar a coluna 'Accident Day', usando mediana.
Sem dados suficientes para imputar a coluna 'Accident DayOfWeek', usando mediana.


/Users/joaosantos/Documents/GitHub/DSAA_ML_41/code/models/helper_functions.py:63: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[target_column].fillna(data[target_column].median(), inplace=True)
/Users/joaosantos/Documents/GitHub/DSAA_ML_41/code/models/helper_functions.py:63: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we a

Sem dados suficientes para imputar a coluna 'Accident DayOfWeek', usando mediana.


/Users/joaosantos/Documents/GitHub/DSAA_ML_41/code/models/helper_functions.py:63: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[target_column].fillna(data[target_column].median(), inplace=True)


KeyError: 'A'

In [ ]:
submission.nunique()